In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


def google_api():
    import time
    import datetime
     #query send to panoply
    scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("#################", scope)
    client = gspread.authorize(creds)
    spreadsheetName = "#################"
    sheetName = "#################"  # <--- please set the sheet name here.
    spreadsheet = client.open(spreadsheetName)
    sheet = spreadsheet.worksheet(sheetName)
    data = sheet.get_all_records()
    tmp_df=pd.DataFrame(data)
    #extract partial strings as needed 
    influencer_df=tmp_df[tmp_df['Platform Used'].str.contains('Instagram')]['Handle/Name']
    influencer_df=influencer_df.replace('@','',regex=True).replace(' ','',regex=True)  
    influencer_df=influencer_df.str.split('//').str[0]
    influencer_df=influencer_df.str.split('/').str[0]
    return influencer_df
    


# In[2]:


def post_within_15days():
    #query data from panoply
    post_table = '#################'
    selFields_post = '*'
    date_range_post_table=' where date >= DATEADD(day,-15, GETDATE())'
    post_query = 'SELECT '+ selFields_post + ' FROM ' + post_table + date_range_post_table+';'

    post = pd.read_sql_query(post_query, cnx)

    return post


# In[3]:


def insta_scrapper(link,insta_sessionid):
    import requests
    out_list=[]
    #sessionID unique for different instagrame ID
    SESSIONID =insta_sessionid
    #header which specific the machine which applys to the request
    headers = {"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36 Edg/87.0.664.57",
           "cookie": f"sessionid={SESSIONID};"}
    #dynamic linktake hastag name
    link=link.strip() 
    base_url = f'{link}?__a=1'
    print(base_url)
    response = requests.get(base_url,headers=headers)
    response = json.loads(response.text)
    #print(base_url)
    print(len(response))
    if len(response)>=1:
        out_list.append(link)
        out_list.append(response['graphql']['shortcode_media']['owner']['username'])
        out_list.append(dt.datetime.fromtimestamp(response['graphql']['shortcode_media']['taken_at_timestamp']).strftime('%Y-%m-%d'))
        out_list.append(response['graphql']['shortcode_media']['edge_media_preview_comment']['count'])
        out_list.append(response['graphql']['shortcode_media']['edge_media_preview_like']['count'])
    return out_list


# In[4]:


def uploader(df,datatable,if_exists):
    #credentials
    POSTGRES_ADDRESS = 'db.panoply.io'
    POSTGRES_PORT = '5439'
    POSTGRES_USERNAME = '#################'
    POSTGRES_PASSWORD = '#################'
    POSTGRES_DBNAME = '#################'
    #data base form
    postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.
                    format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD, 
                        ipaddress=POSTGRES_ADDRESS, 
                        port=POSTGRES_PORT, 
                        dbname=POSTGRES_DBNAME))
    #connect and creat engine
    cnx = create_engine(postgres_str)
    #inject dataframe to datatable
    df.to_sql(datatable, con=cnx, if_exists=if_exists,index=False)


# In[14]:


if __name__ == '__main__':
    import datetime
    today = datetime.date.today()
    first = today.replace(day=1)
    if (today-first).days==14 or (today-first).days==29:#twice a month


        import datetime
        import logging


        import pandas as pd
        from dateutil.relativedelta import relativedelta
        import numpy as np
        import datetime as dt
        import time
        import random
        
        
        from datetime import timedelta, datetime
        import sqlalchemy
        from sqlalchemy import create_engine

        import json
        import requests
        import pandas as pd
        from six import string_types
        from six.moves.urllib.parse import urlencode, urlunparse  # noqa

        from datetime import date
        from datetime import timedelta

        import gspread
        from oauth2client.service_account import ServiceAccountCredentials
        from pprint import pprint
        from datetime import datetime
        import custome_credential
        POSTGRES_ADDRESS = custome_credential.POSTGRES_ADDRESS
        POSTGRES_PORT = custome_credential.POSTGRES_PORT
        POSTGRES_USERNAME = custome_credential.POSTGRES_USERNAME
        POSTGRES_PASSWORD = custome_credential.POSTGRES_PASSWORD
        POSTGRES_DBNAME = custome_credential.POSTGRES_DBNAME
        #connect to database
        postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
            username=POSTGRES_USERNAME,
            password=POSTGRES_PASSWORD,
            ipaddress=POSTGRES_ADDRESS,
            port=POSTGRES_PORT,
            dbname=POSTGRES_DBNAME))

        cnx = create_engine(postgres_str)
        
        #get data from functions
        influencer_df=google_api()
        post=post_within_15days()
        #select id not shown in the influence sheet
        out_df=post[~post['id'].isin(influencer_df)]
        out_df['is_recorded']=0
        df_list=[]
        import instagram_session_id
        #look with sleep to extract data by multiple session id to avoid block by instagram
        for i in list(out_df['link']):
            tmp=insta_scrapper(link=i,insta_sessionid=random.choice(instagram_session_id.instagram_session_id_list))
            print(tmp)
            time.sleep(random.uniform(1, 3)) 
            df_list.append(tmp)
        # filter empty row out due to issue for certain posts
        df_list=[x for x in df_list if x != []]
        df=pd.DataFrame(np.array(df_list),columns=['link', 'id','date','comment_count','like_count'])
        import datetime
        # adjust the datatype to feed in panoply table
        df['update_time']=datetime.date.today()
        df=df.astype({'like_count': 'int32','comment_count': 'int32'})
        df['date']=pd.to_datetime(df['date'])
        df['is_recorded']=0
        datatable='#################'
        if_exists='append'
        uploader(df=df,datatable=datatable,if_exists=if_exists)



    else:
        print('not mid or end day of this month')#modify to day of week for deployment

